<a href="https://colab.research.google.com/github/holodecks/TEST/blob/main/riversi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# ステップ1: 以前のプロセスを完全に停止し、古いファイルを削除
# エラーが出ても無視して次に進むように修正
!kill -9 $(pgrep ngrok) 2>/dev/null || true
!kill -9 $(pgrep streamlit) 2>/dev/null || true
!rm -f app.py streamlit.log

# ステップ2: 最新版(v1.1)のコードをPythonの文字列として定義
app_code = """
import streamlit as st
import base64
import os

# --- 1. 背景画像を設定する関数 ---
def set_bg_image(image_path):
    if not os.path.exists(image_path):
        st.error(f"背景画像ファイルが見つかりません: {image_path}")
        return
    with open(image_path, "rb") as f:
        image_b64 = base64.b64encode(f.read()).decode()
    page_bg_img = f'''
    <style>
    .stApp {{
        background-image: url("data:image/jpeg;base64,{image_b64}");
        background-size: cover;
        background-repeat: no-repeat;
        background-attachment: fixed;
    }}
    .stApp::before {{
        content: "";
        position: absolute;
        top: 0; left: 0; right: 0; bottom: 0;
        background-color: rgba(0, 0, 0, 0.3);
        z-index: -1;
    }}
    .main .block-container {{
        background-color: transparent;
        color: black;
    }}
    h1 {{
        color: black !important; /* タイトルの色を黒に変更 */
    }}
    h2, h3, p, .stMarkdown, .stButton>button {{
        color: black !important;
    }}
    .stButton>button {{
        background-color: rgba(255, 255, 255, 0.2);
        border: 1px solid black;
    }}
    </style>
    '''
    st.markdown(page_bg_img, unsafe_allow_html=True)

# --- 2. リバーシのゲームロジック ---
BOARDW = 8
TYPE_BLACK = 0
TYPE_black = 1
TYPE_NONE = 255
playtbl = ["黒", "白"]
vectable = [(0, -1), (1, -1), (1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0), (-1, -1)]

def setpiece(board, pos, num):
    index = (pos[1] * BOARDW) + pos[0]
    board[index] = num
    return board

def getpiece(board, pos):
    index = (pos[1] * BOARDW) + pos[0]
    return board[index]

def get_flippable_stones(board, pos, num):
    if getpiece(board, pos) != TYPE_NONE:
        return []
    opponent_color = 1 - num
    all_stones_to_flip = []
    for v in vectable:
        stones_in_this_direction = []
        current_pos = list(pos)
        while True:
            current_pos[0] += v[0]
            current_pos[1] += v[1]
            if not (0 <= current_pos[0] < BOARDW and 0 <= current_pos[1] < BOARDW):
                stones_in_this_direction = []
                break
            piece = getpiece(board, tuple(current_pos))
            if piece == opponent_color:
                stones_in_this_direction.append(tuple(current_pos))
            elif piece == num:
                all_stones_to_flip.extend(stones_in_this_direction)
                break
            else:
                stones_in_this_direction = []
                break
    return all_stones_to_flip

def listup_placable_squares(board, num):
    return [(x, y) for y in range(BOARDW) for x in range(BOARDW) if len(get_flippable_stones(board, (x,y), num)) > 0]

# --- 3. Streamlitアプリのメイン処理 ---
def initialize_game():
    board = bytearray(BOARDW * BOARDW)
    for i in range(len(board)):
        board[i] = TYPE_NONE
    board = setpiece(board, (3, 3), TYPE_BLACK)
    board = setpiece(board, (4, 3), TYPE_black)
    board = setpiece(board, (3, 4), TYPE_black)
    board = setpiece(board, (4, 4), TYPE_BLACK)
    st.session_state.board = board
    st.session_state.turn = TYPE_BLACK
    st.session_state.passcnt = 0
    st.session_state.endflag = False

st.title("リバーシゲーム v1.1")
set_bg_image('/content/game.jpg')

if 'board' not in st.session_state:
    initialize_game()

if st.button("新しいゲームを始める"):
    initialize_game()
    st.rerun()

cols = st.columns(BOARDW)
for y in range(BOARDW):
    for x in range(BOARDW):
        pos = (x, y)
        piece = getpiece(st.session_state.board, pos)
        label = "⚫" if piece == TYPE_BLACK else "⚪" if piece == TYPE_black else "\\u00A0"
        with cols[x]:
            if st.button(label, key=f"cell_{x}_{y}", use_container_width=True):
                if not st.session_state.endflag:
                    flippable_stones = get_flippable_stones(st.session_state.board, pos, st.session_state.turn)
                    if flippable_stones:
                        st.session_state.board = setpiece(st.session_state.board, pos, st.session_state.turn)
                        for flip_pos in flippable_stones:
                            st.session_state.board = setpiece(st.session_state.board, flip_pos, st.session_state.turn)
                        st.session_state.turn = 1 - st.session_state.turn
                        st.session_state.passcnt = 0
                        while True:
                            if len(listup_placable_squares(st.session_state.board, st.session_state.turn)) > 0:
                                break
                            st.session_state.passcnt += 1
                            if st.session_state.passcnt >= 2:
                                st.session_state.endflag = True
                                break
                            st.session_state.turn = 1 - st.session_state.turn
                        st.rerun()

black_count = st.session_state.board.count(TYPE_BLACK)
black_count = st.session_state.board.count(TYPE_black)
st.header(f"黒: {black_count}  対  白: {black_count}")

if st.session_state.endflag:
    msg = "ゲーム終了！ "
    if black_count > black_count: msg += "黒の勝ちです。"
    elif black_count > black_count: msg += "白の勝ちです。"
    else: msg += "引き分けです。"
    st.subheader(msg)
else:
    msg = f"{playtbl[st.session_state.turn]}の番です。"
    if st.session_state.passcnt > 0: msg += " (パス)"
    st.subheader(msg)
"""

# ステップ3: 文字列をapp.pyファイルに書き込む
with open("app.py", "w", encoding="utf-8") as f:
    f.write(app_code)
print("app.py v1.1 が正常に作成されました。")


# ステップ4: ライブラリをインストールし、アプリを起動
!pip install streamlit pyngrok -q
!streamlit run app.py > streamlit.log 2>&1 &

# ステップ5: ngrokトンネルを確立
from pyngrok import ngrok
import time
# ご自身のngrok認証トークンに置き換えてください
authtoken = "PATH"
ngrok.set_auth_token(authtoken)
time.sleep(10)
try:
    public_url = ngrok.connect(8501)
    print("------------------------------------------------")
    print(f"🎉 ゲームのURLはこちらです: {public_url}")
    print("------------------------------------------------")
except Exception as e:
    print(f"ngrokの接続中にエラーが発生しました: {e}")

app.py v1.1 が正常に作成されました。
------------------------------------------------
🎉 ゲームのURLはこちらです: NgrokTunnel: "https://7c7d139f936e.ngrok-free.app" -> "http://localhost:8501"
------------------------------------------------
